In [36]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [37]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import roc_auc_score

In [38]:
train = pd.read_csv("train_s3TEQDk.csv")
test = pd.read_csv("test_mSzZ8RL.csv")

In [39]:
train.shape,test.shape

((245725, 11), (105312, 10))

In [40]:
train.isnull().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         29325
Avg_Account_Balance        0
Is_Active                  0
Is_Lead                    0
dtype: int64

In [41]:
test.isnull().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         12522
Avg_Account_Balance        0
Is_Active                  0
dtype: int64

In [42]:
print(train['Credit_Product'].value_counts())
print('******************************************')
print(test['Credit_Product'].value_counts())

No     144357
Yes     72043
Name: Credit_Product, dtype: int64
******************************************
No     61608
Yes    31182
Name: Credit_Product, dtype: int64


In [43]:
#Imputing with Mode
train['Credit_Product']= train['Credit_Product'].dropna()
test['Credit_Product']= test['Credit_Product'].dropna()

In [44]:
train = train.drop(["ID","Gender","Region_Code","Occupation","Channel_Code"],axis=1)
test = test.drop(["ID","Gender","Region_Code","Occupation","Channel_Code"],axis=1)

In [45]:
train.columns

Index(['Age', 'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active',
       'Is_Lead'],
      dtype='object')

In [46]:
test.columns

Index(['Age', 'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active'], dtype='object')

In [47]:
le = LabelEncoder()
var_mod = train.select_dtypes(include='object').columns
for i in var_mod:
    train[i] = le.fit_transform(train[i])
    
for i in var_mod:
    test[i] = le.fit_transform(test[i])

In [48]:
# Seperate Features and Target
X= train.drop(columns = ['Is_Lead'], axis=1)
y= train['Is_Lead']

In [49]:
# 20% data as validation set
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.1,random_state=42)

In [50]:
scaler = MinMaxScaler(feature_range=(0,1))

In [51]:
scaledtrainX = scaler.fit_transform(X_train)
scaledtestX = scaler.fit_transform(X_valid)
scaledx= scaler.fit_transform(X)

scaledtest= scaler.fit_transform(test)



In [52]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from xgboost import XGBClassifier

In [53]:
algos = [  
          LogisticRegression(),
         DecisionTreeClassifier(max_depth=3), 
         XGBClassifier(learning_rate=0.01, n_estimators=25, max_depth=15,gamma=0.6, subsample=0.52,colsample_bytree=0.6,seed=27, 
                    reg_lambda=2, booster='dart', colsample_bylevel=0.6, colsample_bynode=0.5),
         RandomForestClassifier(max_depth=2, random_state=0),
         GaussianNB()
        ]

algos2 = [GradientBoostingRegressor(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=0),
           KNeighborsRegressor(),
          Lasso(), Ridge(), LinearRegression()
         ]

names = [  
         'K Neighbors Regressor', 'Decision Tree Regressor', 'XGB Classifier','Random Forest Classifier',
         'GaussianNB()',
        ]
names2 = ['GBR','KNN','LASSO','RIDGE','Linear REgression']
roc_auc_score_list = []


In [54]:
for name in algos:
    model = name.fit(scaledtrainX,y_train)
    roc= roc_auc_score(y_valid, model.predict_proba(scaledtestX)[:, 1])
    print(name,roc)
    

LogisticRegression() 0.8500987006763038
DecisionTreeClassifier(max_depth=3) 0.8441537453733152
[15:34:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=0.6,
              colsample_bynode=0.5, colsample_bytree=0.6, gamma=0.6, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=25, n_jobs=4, num_parallel_tree=1, random_state=27,
              reg_alpha=0, reg_lambda=2, scale_pos_weight=1, seed=27,
              subsample=0.52, tree_method='exact', validate_parameters=1,
              ver

In [55]:
clf3 = GradientBoostingRegressor(n_estimators=105, learning_rate=0.9,
                                 max_depth=1, random_state=22).fit(scaledtrainX,y_train)
roc_auc_score(y_valid, clf3.predict(scaledtestX),multi_class='ovo')

0.8572723438830566

In [22]:
for name in algos2:
    model = name.fit(scaledtrainX,y_train)
    roc= roc_auc_score(y_valid, model.predict(scaledtestX),multi_class='ovo')
    print(name,roc)
    

GradientBoostingRegressor(learning_rate=1.0, max_depth=1, random_state=0) 0.858128038379531
KNeighborsRegressor() 0.8096004528050988
Lasso() 0.5
Ridge() 0.8501120870302863
LinearRegression() 0.8501113304445513



GBR HAS HIGHEST ROC SCORE

In [42]:
submission01 = pd.read_csv('sample_submission_eyYijxG.csv')
model = GradientBoostingRegressor(n_estimators=105, learning_rate=0.9,
                                 max_depth=1, random_state=22)
model.fit(scaledx, y)
final_predictions = model.predict(scaledtest)
submission01['Is_Lead'] = final_predictions
#only positive predictions for the target variable
submission01['Is_Lead'] = submission01['Is_Lead'].apply(lambda x: 0 if x<0 else x)
submission01.to_csv('my_submission000F.csv', index=False)

In [45]:
# submission01 = pd.read_csv('sample_submission_eyYijxG.csv')
# model = LogisticRegression()
# model.fit(scaledx, y)
# final_predictions = model.predict(scaledtest)
# submission01['Is_Lead'] = final_predictions
# #only positive predictions for the target variable
# submission01['Is_Lead'] = submission01['Is_Lead'].apply(lambda x: 0 if x<0 else x)
# submission01.to_csv('my_submission000LGR.csv', index=False)